In [4]:
import os
import pandas as pd

In [1]:
! wget -O cdetector.tar.gz --no-check-certificate "https://onedrive.live.com/download?cid=05F351192BFCCB72&resid=5F351192BFCCB72%21329196&authkey=ABuqKEi4TX14KbY"

--2022-09-04 14:34:13--  https://onedrive.live.com/download?cid=05F351192BFCCB72&resid=5F351192BFCCB72%21329196&authkey=ABuqKEi4TX14KbY
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://1hanya.db.files.1drv.com/y4mNW82l9JasfKE_ppO8oPHa9_STjI7Z49-UVEJImXjHi-U_griCe_EsKMUwIjyqP9RUjri80nR26pm8mFzlczsNVvN4BQz_40Q8dTST8Zn_Pr5W-_P5REbokTott9BtwPcgEldAeK_AF4e97lhy1vHDVw6Yjo0t7DKGKZGg-dGq4dPZUXhrBaictnE5C_Qbb7m7M_DZ-aInAZr1n8-XP_tQQ/cdetector.tar.gz?download&psid=1 [following]
--2022-09-04 14:34:13--  https://1hanya.db.files.1drv.com/y4mNW82l9JasfKE_ppO8oPHa9_STjI7Z49-UVEJImXjHi-U_griCe_EsKMUwIjyqP9RUjri80nR26pm8mFzlczsNVvN4BQz_40Q8dTST8Zn_Pr5W-_P5REbokTott9BtwPcgEldAeK_AF4e97lhy1vHDVw6Yjo0t7DKGKZGg-dGq4dPZUXhrBaictnE5C_Qbb7m7M_DZ-aInAZr1n8-XP_tQQ/cdetector.tar.gz?download&psid=1
Resolving 1hanya.db.files.1drv.com (1hanya.db.files

In [2]:
! tar xvfz cdetector.tar.gz

test.json
test.zip
train.json
train.zip


In [3]:
! unzip -d cdetector test.zip
! unzip -d cdetector train.zip

Streaming output truncated to the last 5000 lines.
  inflating: cdetector/train/05470.bmp  
  inflating: cdetector/train/04508.bmp  
  inflating: cdetector/train/04271.bmp  
  inflating: cdetector/train/03529.bmp  
  inflating: cdetector/train/01700.bmp  
  inflating: cdetector/train/02587.bmp  
  inflating: cdetector/train/01046.bmp  
  inflating: cdetector/train/01552.bmp  
  inflating: cdetector/train/04408.bmp  
  inflating: cdetector/train/06966.bmp  
  inflating: cdetector/train/00713.bmp  
  inflating: cdetector/train/01192.bmp  
  inflating: cdetector/train/02678.bmp  
  inflating: cdetector/train/06108.bmp  
  inflating: cdetector/train/00858.bmp  
  inflating: cdetector/train/01645.bmp  
  inflating: cdetector/train/07402.bmp  
  inflating: cdetector/train/01206.bmp  
  inflating: cdetector/train/01381.bmp  
  inflating: cdetector/train/06265.bmp  
  inflating: cdetector/train/01951.bmp  
  inflating: cdetector/train/00596.bmp  
  inflating: cdetector/train/05580.bmp  
  infl

In [5]:
mapper = {0 : 'NILM',
          1 : 'ASC-US',
          2 : 'ASC-H',
          3 : 'LSIL',
          4 : 'HSIL',
          5 : 'SCC',
          }

In [6]:
cd_test_path = '/content/cdetector/test'
cd_train_path = '/content/cdetector/train'

In [7]:
# transform the train.json and test.json (CDetector dataset)
"""
Transform the labels of CDetector dataset from cell-level to slide level.

1. actinomyces, herps, flora, candida, and trichomonas changed to NILM. 
2. Combined AGC and ASC-US into a single class.
3. re-order the classes according to the Bethesda system (from bad to worst)
4. create the slide label according to the max number (the worst case) among all cells in the slide

"""
import json
import pandas as pd


with open('train.json', 'r') as j:
     cdetector_train = json.loads(j.read())

with open('test.json', 'r') as j:
     cdetector_test = json.loads(j.read())

cdetector_train_df = pd.DataFrame(cdetector_train['annotations'])
cdetector_test_df = pd.DataFrame(cdetector_test['annotations'])

def transform(df, path):
  return (df
          .assign(
            image_id = df.image_id.apply(lambda fname: os.path.join(path, fname)),
            category_id = df.category_id.apply(lambda label: mapper[label])
          )
        )
  
cd_train = (cdetector_train_df
 [['image_id', 'category_id']]
 .replace({6:2, 11:0, 10:0, 9:0, 8:0, 7:0})
 .groupby('image_id').agg('max')
 .reset_index()
 .pipe(transform, cd_train_path)
)

cd_test = (cdetector_test_df
 [['image_id', 'category_id']]
 .replace({6:2, 11:0, 10:0, 9:0, 8:0, 7:0})
 .groupby('image_id').agg('max')
 .reset_index()
 .pipe(transform, cd_test_path)
)

In [10]:
#original mapping
cdetector_train['categories']

[{'id': 1, 'supercategory': 'none', 'name': 'ascus'},
 {'id': 2, 'supercategory': 'none', 'name': 'asch'},
 {'id': 3, 'supercategory': 'none', 'name': 'lsil'},
 {'id': 4, 'supercategory': 'none', 'name': 'hsil'},
 {'id': 5, 'supercategory': 'none', 'name': 'scc'},
 {'id': 6, 'supercategory': 'none', 'name': 'agc'},
 {'id': 7, 'supercategory': 'none', 'name': 'trichomonas'},
 {'id': 8, 'supercategory': 'none', 'name': 'candida'},
 {'id': 9, 'supercategory': 'none', 'name': 'flora'},
 {'id': 10, 'supercategory': 'none', 'name': 'herps'},
 {'id': 11, 'supercategory': 'none', 'name': 'actinomyces'}]

In [9]:
#combining the train and test into a single dataset
cd_dataset = pd.concat([cd_train, cd_test], axis=0)

In [26]:
cd_dataset.category_id.value_counts().reset_index()

,index,category_id
0,HSIL,3462
1,ASC-H,1287
2,NILM,1104
3,LSIL,644
4,ASC-US,566
5,SCC,347
